# News event detection project part 1
## Trigger detection



In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')


Mounted at /content/gdrive


In [ ]:
import pandas as pd
df = pd.read_csv("/content/gdrive/My Drive/ner dataset/ner_dataset.csv", encoding="latin-1")

In [ ]:
df.head()

,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,NaN,of,IN,O
2,NaN,demonstrators,NNS,O
3,NaN,have,VBP,O
4,NaN,marched,VBN,O


In [ ]:
!pip install pkuseg

     |████████████████████████████████| 50.2MB 61kB/s 


In [ ]:
# !pip install 

In [ ]:
!pip install transformers

     |████████████████████████████████| 778kB 5.5MB/s 
     |████████████████████████████████| 3.0MB 34.4MB/s 
     |████████████████████████████████| 1.1MB 14.2MB/s 
     |████████████████████████████████| 890kB 47.7MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=85fbc4d617fd94106c4616714e256aa5d699ed398747545c86754deebae1b8fa
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
import transformers

MAX_LEN = 128
TRAIN_BATCH_SIZE = 32
VALID_BATCH_SIZE = 8
EPOCHS = 5
BASE_MODEL_PATH = "bert-base-chinese"
MODEL_PATH = "ner_model.bin"
TRAINING_FILE = '/content/gdrive/My Drive/事件抽取挑战赛_事件抽取初赛数据/事件抽取挑战赛_事件抽取初赛数据/train/train.csv'
TOKENIZER = transformers.BertTokenizer.from_pretrained(
    BASE_MODEL_PATH,
    do_lower_case=True
)

In [ ]:
import pandas as pd
import numpy as np
import pkuseg

from sklearn import preprocessing
from collections import defaultdict

def re_order(data):
    """
    Reset id of each sentence, make duplicated sentences have a single id.
    :param data: pd.dataframe
    :return: none
    """
    data.reset_index()
    i = j = 0
    while i < len(data):
        if data['news'].duplicated()[i]:
            i += 1
        else:
            i += 1
            j = data['id']
        data['id'] = j

def merge_text(data):
    """
    Merge same sentence into one row, save different triggers/objects/subjects/times/locations into lists correspondingly
    :param data: pd.dataframe
    :return: a nested dictionary
    """
    dict = defaultdict(lambda: defaultdict())
    for ind, row in data.iterrows():
        if row['id'] not in dict:
            dict[row['id']]['news'] = row['news']
            dict[row['id']]['trigger'] = [row['trigger']]
            dict[row['id']]['object'] = [row['object']]
            dict[row['id']]['subject'] = [row['subject']]
            if not pd.isnull(row['time']):
                dict[row['id']]['time'] = [row['time']]
            else:
                dict[row['id']]['time'] = ['NaN']
            if not pd.isnull(row['location']):
                dict[row['id']]['location'] = [row['location']]
            else:
                dict[row['id']]['location'] = ['NaN']
        else:
            if row['trigger'] not in dict[row['id']]['trigger']:
                dict[row['id']]['trigger'].append(row['trigger'])
            if row['object'] not in dict[row['id']]['object']:
                dict[row['id']]['object'].append(row['object'])
            if row['subject'] not in dict[row['id']]['subject']:
                dict[row['id']]['subject'].append(row['subject'])
            if not pd.isnull(row['time']):
                if row['time'] not in dict[row['id']]['time']:
                    dict[row['id']]['time'].append(row['time'])
            if not pd.isnull(row['location']):
                if row['location'] not in dict[row['id']]['location']:
                    dict[row['id']]['location'].append(row['location'])
    return dict


def label_text(dict):
    """
    Labeling each phrase or word follows BOI rule. There are 11 different labels:
    1. B-trigger, I-trigger
    2. B-object, I-object
    3. B-subject, I-subject
    4. B-location, I-location
    5. B-time, I-time
    6. O: Else
    :param d: dictionary of full dataset
    :return: label sequence
    """
    TotalLabels = []
    #     print(d.keys())
    for ind, key in enumerate(dict.keys()):
        #         print(key)
        text = dict[key]['news']
        #         print(text)
        labels = ['O' for word in text]
        try:
            # find start location of trigger
            for trigger in dict[key]['trigger']:
                start = text.find(trigger)
                if start == -1:
                    continue
                labels[start] = 'B-trigger'
                for loc in range(start + 1, start + len(trigger)):
                    labels[loc] = 'I-trigger'
        except:
            pass
        try:
            # find start location of object
            for obj in dict[key]['object']:
                start = text.find(obj)
                if start == -1:
                    continue
                labels[start] = 'B-object'
                for loc in range(start + 1, start + len(obj)):
                    labels[loc] = 'I-object'
        except:
            pass
        try:
            # find start location of subject
            for subject in dict[key]['subject']:
                start = text.find(subject)
                if start == -1:
                    continue
                labels[start] = 'B-subject'
                for loc in range(start + 1, start + len(subject)):
                    labels[loc] = 'I-subject'
        except:
            pass
        try:
            # find start location of time
            for time in dict[key]['time']:
                start = text.find(time)
                if start == -1:
                    continue
                labels[start] = 'B-time'
                for loc in range(start + 1, start + len(time)):
                    labels[loc] = 'I-time'
        except:
            pass

        try:
            # find start location of location
            for location in dict[key]['location']:
                start = text.find(location)
                if start == -1:
                    continue
                labels[start] = 'B-location'
                for loc in range(start + 1, start + len(location)):
                    labels[loc] = 'I-location'
        except:
            pass
        TotalLabels.append(labels)

    return TotalLabels


def word_segment(dic, TotalLabels):
    """
    Segment sentences into words by using pkuseg.
    Then on character level:
    Label each char with its boundaries within a word by BMES. B stands for begin, M middle, E end, S single.
    Label each char with its POS tags
    Lable each char with its Labels from Total Labels
    Return a list combine characters, BMES, POS, Labels
    :param df: dataset dictionary, TotalLabels
    :return: List
    """
    #     # Load Stop words list from Harbin Institution of Technology
    #     with open(stop_words_path, encoding='UTF-8') as f:
    #         stopWordsList = f.read().splitlines()
    #     f.close()

    pos_tags = []
    bounds = []
    text = []
    dataset = []

    seg = pkuseg.pkuseg(postag=True)

    for ind, key in enumerate(dic.keys()):

        #         print('word segment No.{}'.format(ind))

        boundaries = ['M' for word in dic[key]['news']]
        sent = [word for word in dic[key]['news']]
        tags = []

        #         for word, tag in pseg.cut(d[key]['news']):
        for word, tag in seg.cut(dic[key]['news']):

            if len(word) == 1:
                start = len(tags)
                boundaries[start] = 'S'
                tags.append(tag)
            else:
                start = len(tags)
                boundaries[start] = 'B'
                tags += [tag] * len(word)
                end = len(tags) - 1
                boundaries[end] = 'E'

        pos_tags.append(tags)
        bounds.append(boundaries)
        text.append(sent)

        dataset.append(list(zip(*[sent, tags, boundaries, TotalLabels[ind]])))

    return dataset



def process_data(data_path):
    df = pd.read_csv(data_path, encoding='utf-8')
    # df['Sentence #'] = df['Sentence #'].fillna(method='ffill')
    re_order(df)
    d = merge_text(df)
    TotalLabels = label_text(d)
    data = word_segment(d, TotalLabels)

    SentenceNum = []
    Word = []
    POS = []
    BEMS = []
    Tag = []
    for idx, item in enumerate(data):
        for i in item:
            SentenceNum.append(idx)
            Word.append(i[0])
            POS.append(i[1])
            BEMS.append(i[2])
            Tag.append(i[3])

    df = pd.DataFrame(list(zip(*[SentenceNum, Word, POS, BEMS, Tag])),
                      columns=['Sentence #', 'Word', 'POS', 'BEMS', 'Tag'])

    enc_pos = preprocessing.LabelEncoder()
    enc_bems = preprocessing.LabelEncoder()
    enc_tag = preprocessing.LabelEncoder()

    df['POS'] = enc_pos.fit_transform(df['POS'])
    # print(df['POS'])
    df['BEMS'] = enc_bems.fit_transform(df['BEMS'])
    df['Tag'] = enc_tag.fit_transform(df['Tag'])

    sentences = df.groupby('Sentence #')['Word'].apply(list).values
    pos = df.groupby('Sentence #')['POS'].apply(list).values
    bems = df.groupby('Sentence #')['BEMS'].apply(list).values
    tag = df.groupby('Sentence #')['Tag'].apply(list).values

    return sentences, pos, bems, tag, enc_pos, enc_bems, enc_tag

sentences, pos, bems, tag, enc_pos, enc_bems, enc_tag = process_data(TRAINING_FILE)


Downloading: "https://github.com/lancopku/pkuseg-python/releases/download/v0.0.16/postag.zip" to /root/.pkuseg/postag.zip
100%|██████████| 41424981/41424981 [00:01<00:00, 26777338.29it/s]


In [ ]:
print(sentences, pos, bems, tag, enc_pos, enc_bems, enc_tag)

[list(['新', '华', '社', '北', '京', '4', '月', '7', '日', '电', '（', '记', '者', '王', '优', '玲', '）', '人', '力', '资', '源', '社', '会', '保', '障', '部', '7', '日', '启', '动', '第', '七', '届', '大', '中', '城', '市', '联', '合', '招', '聘', '高', '校', '毕', '业', '生', '春', '季', '专', '场', '活', '动'])
 list(['活', '动', '针', '对', '高', '校', '毕', '业', '生', '特', '点', '和', '需', '求', '，', '集', '中', '归', '集', '发', '布', '一', '批', '适', '合', '高', '校', '毕', '业', '生', '的', '就', '业', '岗', '位', '，', '并', '鼓', '励', '引', '导', '供', '需', '双', '方', '应', '用', '高', '校', '毕', '业', '生', '精', '准', '招', '聘', '平', '台', '精', '准', '匹', '配', '对', '接'])
 list(['图', '集', '4', '月', '8', '日', '，', '三', '大', '运', '营', '商', '中', '国', '移', '动', '、', '中', '国', '电', '信', '和', '中', '国', '联', '通', '将', '联', '合', '举', '行', '线', '上', '发', '布', '会', '，', '发', '布', '5', 'G', '消', '息', '白', '皮', '书', '，', '携', '手', '生', '态', '合', '作', '伙', '伴', '，', '一', '起', '开', '启', '5', 'G', '消', '息', '业', '务', '全', '新', '篇', '章'])
 ...
 list(['星', '岛', '环', '球', '网', '消', '息',

In [ ]:
# # x = '2020年一月'
# # [TOKENIZER.encode(i,add_special_tokens=False)[0] for i in x]
# TOKENIZER.encode(
#     x,
#     add_special_tokens=False
# )

In [ ]:
import torch


class EntityDataset:
    def __init__(self, texts, pos, bems, tags):
        # texts: [["hi", ",", "my", "name", "is", "abhishek"], ["hello".....]]
        # pos/tags: [[1 2 3 4 1 5], [....].....]]
        self.texts = texts
        self.pos = pos
        self.bems = bems
        self.tags = tags
    
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, item):
        text = self.texts[item]
        pos = self.pos[item]
        bems = self.bems[item]
        tags = self.tags[item]

        ids = []
        target_pos = []
        target_bems = []
        target_tag =[]

        for i, s in enumerate(text):
            # inputs = TOKENIZER.encode(
            #     s,
            #     add_special_tokens=False
            # )
            inputs = [TOKENIZER.encode(i,add_special_tokens=False)[0] for i in s]
            input_len = len(inputs)
            ids.extend(inputs)
            target_pos.extend([pos[i]] * input_len)
            target_bems.extend([bems[i]] * input_len)
            target_tag.extend([tags[i]] * input_len)

        ids = ids[:MAX_LEN - 2]
        target_pos = target_pos[:MAX_LEN - 2]
        target_bems = target_bems[:MAX_LEN - 2]
        target_tag = target_tag[:MAX_LEN - 2]

        ids = [101] + ids + [102]
        target_pos = [0] + target_pos + [0]
        target_bems = [0] + target_bems + [0]
        target_tag = [0] + target_tag + [0]

        mask = [1] * len(ids)
        token_type_ids = [0] * len(ids)

        padding_len = MAX_LEN - len(ids)

        ids = ids + ([0] * padding_len)
        mask = mask + ([0] * padding_len)
        token_type_ids = token_type_ids + ([0] * padding_len)
        target_pos = target_pos + ([0] * padding_len)
        target_bems = target_bems + ([0] * padding_len)
        target_tag = target_tag + ([0] * padding_len)

        return {
            "ids": torch.tensor(ids, dtype=torch.long),
            "mask": torch.tensor(mask, dtype=torch.long),
            "token_type_ids": torch.tensor(token_type_ids, dtype=torch.long),
            "target_pos": torch.tensor(target_pos, dtype=torch.long),
            "target_bems": torch.tensor(target_bems, dtype=torch.long),
            "target_tag": torch.tensor(target_tag, dtype=torch.long),
        }

In [ ]:
import joblib
from sklearn.model_selection import train_test_split

meta_data = {
    'enc_pos': enc_pos,
    'enc_bems': enc_bems,
    'enc_tag': enc_tag
}
joblib.dump(meta_data, "meta.bin")
num_pos = len(list(enc_pos.classes_))
num_bems = len(list(enc_bems.classes_))
num_tag = len(list(enc_tag.classes_))

(
    train_sentences,
    test_sentences,
    train_pos,
    test_pos,
    train_bems,
    test_bems,
    train_tag,
    test_tag
) = train_test_split(sentences, pos, bems, tag, random_state=42, test_size=0.1)

train_dataset = EntityDataset(
    texts=train_sentences, pos=train_pos, bems=train_bems, tags=train_tag
)

train_data_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=TRAIN_BATCH_SIZE, num_workers=4
)

valid_dataset = EntityDataset(
    texts=test_sentences, pos=test_pos, bems=test_bems, tags=test_tag
)

valid_data_loader = torch.utils.data.DataLoader(
    valid_dataset, batch_size=VALID_BATCH_SIZE, num_workers=1
)

In [ ]:
next(iter(train_dataset))

{'ids': tensor([ 101, 6858, 6814, 5632,  712, 4777, 1355, 8024,  677, 3862, 2157, 1265,
         7355, 5330, 2972, 1139, 3173, 1501, 4277, 8024, 1429, 2471,  679, 1398,
         2399, 7977, 3667, 3867, 6589, 5442, 8024, 2810, 1920, 2356, 1767,  819,
         7583,  102,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0]),
 'mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1,

In [ ]:
# next(iter(train_data_loader))

In [ ]:
import torch.nn as nn
import torch

def loss_fn(output, target, mask, num_labels):
    l_fn = nn.CrossEntropyLoss()
    active_loss = mask.view(-1) == 1
    active_logits = output.view(-1, num_labels)
    active_labels = torch.where(
        active_loss,
        target.view(-1),
        torch.tensor(l_fn.ignore_index).type_as(target)
    )
    loss = l_fn(active_logits, active_labels)
    return loss


class EntityModel(nn.Module):
    def __init__(self, num_tag, num_bems, num_pos):
        super(EntityModel, self).__init__()
        self.num_tag = num_tag
        self.num_bems = num_bems
        self.num_pos = num_pos
        self.bert = transformers.BertModel.from_pretrained(BASE_MODEL_PATH)
        self.bert_drop_1 = nn.Dropout(0.3)
        self.bert_drop_2 = nn.Dropout(0.3)
        self.bert_drop_3 = nn.Dropout(0.3)
        self.out_tag = nn.Linear(768, self.num_tag)
        self.out_bems = nn.Linear(768, self.num_bems)
        self.out_pos = nn.Linear(768, self.num_pos)
    
    def forward(self, ids, mask, token_type_ids, target_pos, target_bems, target_tag):
        o1, _ = self.bert(ids, attention_mask=mask, token_type_ids=token_type_ids)

        bo_tag = self.bert_drop_1(o1)
        bo_bems = self.bert_drop_2(o1)
        bo_pos = self.bert_drop_3(o1)

        tag = self.out_tag(bo_tag)
        bems = self.out_bems(bo_bems)
        pos = self.out_pos(bo_pos)

        loss_tag = loss_fn(tag, target_tag, mask, self.num_tag)
        loss_bems = loss_fn(bems, target_bems, mask, self.num_bems)
        loss_pos = loss_fn(pos, target_pos, mask, self.num_pos)

        loss = (loss_tag + loss_bems + loss_pos) / 3

        return tag, bems, pos, loss

In [ ]:
device = torch.device('cuda')
model = EntityModel(num_pos=num_pos, num_bems = num_bems, num_tag = num_tag)
model.to(device)

EntityModel(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(21128, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)


In [ ]:
param_optimizer = list(model.named_parameters())
no_decay = ["bias", "LayerNorm.bias", "LayerNorm.weight"]
optimizer_parameters = [
    {
        "params": [
            p for n, p in param_optimizer if not any(nd in n for nd in no_decay)
        ],
        "weight_decay": 0.001,
    },
    {
        "params": [
            p for n, p in param_optimizer if any(nd in n for nd in no_decay)
        ],
        "weight_decay": 0.0,
    },
]

In [ ]:
from transformers import AdamW, get_linear_schedule_with_warmup

num_train_steps = int(len(train_sentences) / TRAIN_BATCH_SIZE * EPOCHS)
optimizer = AdamW(optimizer_parameters, lr=3e-5)
scheduler = get_linear_schedule_with_warmup(
    optimizer, num_warmup_steps=0, num_training_steps=num_train_steps
)

In [ ]:
import torch
from tqdm import tqdm
def train_fn(data_loader, model, optimizer, device, scheduler):
    model.train()
    final_loss = 0
    for data in tqdm(data_loader, total=len(data_loader)):
        for k, v in data.items():
            data[k] = v.to(device)
        optimizer.zero_grad()
        _, _, _, loss = model(**data)
        loss.backward()
        optimizer.step()
        scheduler.step()
        final_loss += loss.item()
    return final_loss / len(data_loader)
def eval_fn(data_loader, model, device):
    model.eval()
    final_loss = 0
    for data in tqdm(data_loader, total=len(data_loader)):
        for k, v in data.items():
            data[k] = v.to(device)
        _, _, _, loss = model(**data)
        final_loss += loss.item()
    return final_loss / len(data_loader)

In [ ]:
best_loss = np.inf
for epoch in range(EPOCHS):
    train_loss = train_fn(train_data_loader, model, optimizer, device, scheduler)
    test_loss = eval_fn(valid_data_loader, model, device)
    print(f"Train Loss = {train_loss} Valid Loss = {test_loss}")
    if test_loss < best_loss:
        torch.save(model.state_dict(), MODEL_PATH)
        torch.save(model,'trigger_model.pt')
        best_loss = test_loss

100%|██████████| 72/72 [00:05<00:00, 12.84it/s]


Train Loss = 0.7311545041976152 Valid Loss = 0.37407245486974716


100%|██████████| 72/72 [00:05<00:00, 12.71it/s]


Train Loss = 0.3602692096689601 Valid Loss = 0.31486657323936623


100%|██████████| 72/72 [00:05<00:00, 12.58it/s]


Train Loss = 0.2955791629206987 Valid Loss = 0.29252462792727685


100%|██████████| 72/72 [00:05<00:00, 12.64it/s]


Train Loss = 0.2605533841767429 Valid Loss = 0.28909531359871227


100%|██████████| 72/72 [00:05<00:00, 12.61it/s]


Train Loss = 0.24123876486663465 Valid Loss = 0.288140366681748


In [ ]:
test_data = pd.read_csv('/content/gdrive/My Drive/事件抽取挑战赛_事件抽取初赛数据/事件抽取挑战赛_事件抽取初赛数据/test/test.csv')
test_data.head()

,id,news
0,7953,此外，深航积极恢复湖北其他城市的航班运力，已于近日陆续恢复襄阳往返深圳、惠州、沈阳、珠海，宜...
1,7955,同时，因其违反中国法律、法规规定，妨害社会管理，根据《出境入境管理法》第八十一条第一款，越秀...
2,7956,2008年间，佛山市南海区某五金机械厂等8名合法债权人因与罗某丰所经营的某灯具公司发生买卖合...
3,7959,广州男子涉嫌偷拍女同学发布淫秽信息，学校发声4月8日9时许，广州警方发布通报，经过缜密调查，...
4,7960,作为广东深圳一家节能企业的董事长，从1月29日开始，李海建便来到武汉参与医疗废弃物焚烧车间建设


In [ ]:
enc_tag


LabelEncoder()

In [ ]:
test

NameError: ignored

In [ ]:
meta_data = joblib.load('meta.bin')
enc_pos = meta_data['enc_pos']
enc_bems = meta_data['enc_bems']
enc_tag = meta_data['enc_tag']

num_tag = len(list(enc_tag.classes_))
num_bems = len(list(enc_bems.classes_))
num_pos = len(list(enc_pos.classes_))


device = torch.device('cuda')
# model = EntityModel(num_tag = num_tag, num_bems = num_bems, num_pos=num_pos)
# model.load_state_dict(torch.load('trigger_model.bin'))
model = torch.load('trigger_model.pt')
model.to(device)

ids = []
news = []
triggers = []
for idx, item in test_data.iterrows():
    # print(f"processing No.{idx} of {len(test_data)}")
    sentence = item['news']
    id = item['id']
    encode_sentence = [TOKENIZER.encode(i,add_special_tokens=False) for i in sentence]
    test = EntityDataset(
        texts=[sentence],
        pos=[[0] * len(sentence)],
        bems=[[0] * len(sentence)],
        tags=[[0] * len(sentence)]
    )
    with torch.no_grad():
        data = test[0]
        for k, v in data.items():
            data[k] = v.to(device).unsqueeze(0)
        tag, bems, pos, _ = model(**data)
        #extract object

        tags = tag.argmax(2).cpu().numpy()[0][1:len(encode_sentence)+1]
        text = [word for word in sentence]
        label_list = []
        location_list = []
        temp_idx = -1
        for idx, (i,j) in enumerate(zip(tags,text)):
            # if i == 1:
            if i == 4:
                temp = j
                # start = idx
                temp_idx = idx
            # if i == 6 and temp_idx != -1:
            if i == 9 and temp_idx != -1:
                temp += j
                temp_idx += 1
            if i not in [4,9] and temp_idx != -1:
                # temp = str(start)+temp
                # temp = list(set(temp))
                label_list.append(temp)
                # location_list.append(temp_loc)
                temp = ''
                # start = -1
                temp_idx = -1
        ids.append(id)
        news.append(sentence)
        # label_list = list(set(label_list))
        triggers.append(label_list)
final = pd.DataFrame(zip(ids,news,triggers),columns=['id','news','trigger'])



In [ ]:
final.head()

,id,news,trigger
0,7953,此外，深航积极恢复湖北其他城市的航班运力，已于近日陆续恢复襄阳往返深圳、惠州、沈阳、珠海，宜...,"[恢复, 恢复]"
1,7955,同时，因其违反中国法律、法规规定，妨害社会管理，根据《出境入境管理法》第八十一条第一款，越秀...,[作出]
2,7956,2008年间，佛山市南海区某五金机械厂等8名合法债权人因与罗某丰所经营的某灯具公司发生买卖合...,"[发生, 申请, 查封]"
3,7959,广州男子涉嫌偷拍女同学发布淫秽信息，学校发声4月8日9时许，广州警方发布通报，经过缜密调查，...,"[发布, 发布, 抓获]"
4,7960,作为广东深圳一家节能企业的董事长，从1月29日开始，李海建便来到武汉参与医疗废弃物焚烧车间建设,[参与]


In [ ]:
import pickle

with open('triggers.pickle', 'wb') as handle:
    pickle.dump(final, handle, protocol=pickle.HIGHEST_PROTOCOL)


In [ ]:
import pickle

with open('triggers.pickle', 'rb') as handle:
    ff = pickle.load(handle)

In [ ]:
final.to_csv('final_object.csv')

In [ ]:
final.head()

,id,news,object
0,7953,此外，深航积极恢复湖北其他城市的航班运力，已于近日陆续恢复襄阳往返深圳、惠州、沈阳、珠海，宜...,[深航]
1,7955,同时，因其违反中国法律、法规规定，妨害社会管理，根据《出境入境管理法》第八十一条第一款，越秀...,[越秀区公安分局]
2,7956,2008年间，佛山市南海区某五金机械厂等8名合法债权人因与罗某丰所经营的某灯具公司发生买卖合...,[佛山市南海区某五金机械厂等8名合法债权人]
3,7959,广州男子涉嫌偷拍女同学发布淫秽信息，学校发声4月8日9时许，广州警方发布通报，经过缜密调查，...,"[广州男子, 广州警方, 广州白云警方]"
4,7960,作为广东深圳一家节能企业的董事长，从1月29日开始，李海建便来到武汉参与医疗废弃物焚烧车间建设,[李海建]
